In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../invert')
import mne
import pickle as pkl
from time import time
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from invert.forward import get_info, create_forward_model
from invert.solvers.esinet import generator
from invert.util import pos_from_forward

pp = dict(surface='white', hemi='both', verbose=0)

# Load Data

In [2]:
with open("forward_model/64ch_info.pkl", "rb") as f:
    info = pkl.load(f)
    
sim_type = "extended"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_ext, x_test_ext, y_test_ext, sim_info_ext, _, _ = pkl.load(f)

sim_type = "single"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_sing, x_test_sing, y_test_sing, sim_info_sing, _, _ = pkl.load(f)



sim_info = pd.concat([sim_info_sing, sim_info_ext])
sim_info["sample"] = sim_info.index.values
sim_info["sample"][500:] += 500

fwd = mne.read_forward_solution("forward_model/64ch_ico3-fwd.fif", verbose=0)
fwd = mne.convert_forward_solution(fwd, force_fixed=True)
pos = pos_from_forward(fwd)
source_model = fwd['src']
vertices = [source_model[0]['vertno'], source_model[1]['vertno']]
distances = cdist(pos, pos)
argsorted_distance_matrix = np.argsort(distances, axis=1)

sim_type = "single"
fn = f"results/results_{sim_type}.pkl"
with open(fn, 'rb') as f:
    results_single = pkl.load(f)

sim_type = "extended"
fn = f"results/results_{sim_type}.pkl"
with open(fn, 'rb') as f:
    results_extended = pkl.load(f)

df_single = pd.DataFrame(results_single)
df_single["Source Extend"] = "Single"
df_extended = pd.DataFrame(results_extended)
df_extended["Source Extend"] = "Extended"

df = pd.concat([df_single, df_extended])

n_methods = len(set(df.Method.values))
sample = np.tile(np.arange(sim_info.shape[0]/2).astype(int), n_methods*2)
df["Sample"] = sample
df["Sample"][df["Source Extend"]=="Extended"] += 500

snrs = []
n_sources = []
for i in range(df.shape[0]):
    sample = df["Sample"].values[i]
    snr = sim_info.loc[sim_info["sample"] == sample]["snr"].values[0]
    n_source = sim_info.loc[sim_info["sample"] == sample]["n_sources"].values[0]
    snrs.append(snr)
    n_sources.append(n_source)
df["SNR"] = snrs
df["Number of sources"] = n_sources

snr_min = df["SNR"].min()
snr_max = df["SNR"].max()
print(snr_min, snr_max)


bins = [[0.1, 10], [10, 50], [50, 100]]
bins_idc = np.arange(len(bins))
bin_names = ["0.1 - 5", "5 - 20", "20 - 100"]
snr_bins_df = []
for i in range(df.shape[0]):
    snr = df["SNR"].values[i]
    for bin_idx, bin in zip(bins_idc, bins):
        if snr < bin[1] and snr >= bin[0]:
            snr_bins_df.append(bin_idx)
            break
df["SNR Bins"] = snr_bins_df

df.Method[df.Method=="Convexity Champagne"] = "Champagne"
df.Method[df.Method=="FLEX-MUSIC"] = "FLEX\nMUSIC"
df.Method[df.Method=="TRAP-MUSIC"] = "TRAP\nMUSIC"
df.Method[df.Method=="eLORETA"] = "eLOR"
df.rename(columns = {col: col.replace("-", " ").replace("_", " ") for col in df.columns}, inplace = True)
df

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]


C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\1917075184.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim_info["sample"][500:] += 500
C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\1917075184.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sample"][df["Source Extend"]=="Extended"] += 500
C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\1917075184.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sample"]

0.1195153899469284 99.88138915103026


C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\1917075184.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Method[df.Method=="Convexity Champagne"] = "Champagne"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\1917075184.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Method[df.Method=="FLEX-MUSIC"] = "FLEX\nMUSIC"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\1917075184.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

,Mean Squared Error,Normalized Mean Squared Error,Mean Localization Error,AUC,Corr,EMD,Sparsity pred,Sparsity true,Active True,Active Pred,Method,Time,Source Extend,Sample,SNR,Number of sources,SNR Bins
0,3.829238e-07,5.306143e-04,3.261167,0.639031,0.799351,60391.698086,1.830591,2.143009,0.006231,0.007009,FLEX\nMUSIC,1.305580,Single,0,22.225641,8,1
1,5.683600e-07,7.302426e-04,0.000000,0.937500,0.819352,29862.010565,2.566176,2.621378,0.006231,0.005452,FLEX\nMUSIC,0.639510,Single,1,38.440601,8,1
2,1.623973e-08,4.496306e-06,0.000000,0.750000,0.997126,11507.457389,1.000000,1.319880,0.001558,0.000779,FLEX\nMUSIC,0.313382,Single,2,19.230628,2,1
3,1.668565e-08,7.358029e-07,0.000000,0.750000,0.999528,5711.726314,1.000000,1.401588,0.001558,0.000779,FLEX\nMUSIC,0.298180,Single,3,69.787060,2,2
4,2.861071e-07,3.314691e-04,5.310500,1.000000,0.861249,22743.398146,2.051503,2.005859,0.003894,0.003894,FLEX\nMUSIC,0.574463,Single,4,35.832048,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2.167456e-06,5.666815e-02,16.812219,0.598777,-0.003170,180119.165319,30.161875,5.036119,0.056854,1.000000,MCMV,1.527125,Extended,995,97.656203,7,2
2496,1.725461e-06,2.449491e-02,19.013159,0.514655,0.008477,181836.877946,29.140726,5.910116,0.073209,1.000000,MCMV,1.689509,Extended,996,34.616781,7,1
2497,9.333329e-07,1.100789e-02,18.430904,0.575150,-0.007718,175807.498060,29.241081,4.311293,0.038162,1.000000,MCMV,1.629163,Extended,997,99.714634,7,2
2498,7.445693e-06,1.276045e-01,24.104887,0.519789,-0.018491,183955.241780,31.144250,5.035551,0.052181,1.000000,MCMV,1.573825,Extended,998,1.186548,7,0


# Box- and Scatter- Plots

In [3]:
sns.set(style="whitegrid", font_scale=1.)

tick_params = dict(
    axis='y',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    left=True,      # ticks along the bottom edge are off
    right=False,         # ticks along the top edge are off
    labelbottom=False
)

medianprops = {
    "linewidth": 2,
    "linestyle": "dashed"
    }

order = ['FLEX\nMUSIC', 'TRAP\nMUSIC', 'Champagne', 'MCMV', 'eLOR']

#================================================================================#

fig1 = plt.figure(figsize=(16,5))
plt.subplot(131)
sns.boxplot(data=df, x="Method", y="Mean Localization Error", hue="Source Extend", order=order, medianprops=medianprops)
plt.tick_params(**tick_params)
plt.ylim(-3, 43)
plt.ylabel("Mean Localization Error [mm]")
plt.gca().get_legend().remove()

plt.subplot(132)
g = sns.boxplot(data=df, x="Method", y="EMD", hue="Source Extend", order=order, medianprops=medianprops)
plt.tick_params(**tick_params)
plt.gca().get_legend().remove()

plt.subplot(133)
sns.boxplot(data=df, x="Method", y="Mean Squared Error", hue="Source Extend", order=order, medianprops=medianprops)
plt.ylim(-0.05e-5, 1e-5)
plt.tick_params(**tick_params)

plt.tight_layout(pad=2)
df.rename(columns={"Sparsity pred": "Sparsity"}, inplace=True)

#================================================================================#

fig2 = plt.figure(figsize=(10,5))
plt.subplot(121)
sns.boxplot(data=df, x="Method", y="Sparsity", hue="Source Extend", order=order, medianprops=medianprops)
plt.tick_params(**tick_params)
plt.gca().get_legend().remove()

plt.subplot(122)
g = sns.boxplot(data=df, x="Method", y="Time", hue="Source Extend", order=order, medianprops=medianprops)
g.set_yscale("log")
plt.ylabel("Computation Time [s]")
plt.tick_params(**tick_params)

plt.tight_layout(pad=2)

from scipy.stats import pearsonr
sns.set(style="white", font_scale=1.)
tick_params = dict(
    axis='both',          # changes apply to the y-axis
    which='both',      # both major and minor ticks are affected
    direction="inout",
    left=True,      # ticks along the bottom edge are off
    bottom=True,
    right=False,         # ticks along the top edge are off
    labelbottom=True
)

#================================================================================#

fig3 = plt.figure(figsize=(8, 6))
for i, method in enumerate(order):
    print(method)
    plt.subplot(2,3,i+1)
    df_temp = df[df.Method==method]

    a = df_temp["Sparsity true"].values
    b = df_temp["Sparsity"].values
    nans = (np.isnan(a) | np.isnan(b))
    a[nans] = np.nanmedian(a)
    b[nans] = np.nanmedian(b)
    r, p = pearsonr(a, b)
    # print(f"{method} sparsity: r = {r:.2f}")

    
    sns.scatterplot(data=df_temp, y="Sparsity", x="Sparsity true")
    plt.gca().set_aspect('equal', adjustable='box')

    

    maxval = np.max( [plt.ylim()[1], plt.xlim()[1]])
    plt.ylim(0,  maxval)
    plt.xlim(0, maxval)
    
    
    plt.plot([0, maxval], [0, maxval], 'k')
    plt.ylabel("Predicted Sparsity")
    plt.xlabel("True Sparsity")
    plt.title(method)
    r_text = f"r = {r:.2f}"
    if p<0.001:
        r_text += " ***"
    elif p<0.01:
        r_text += " **"
    elif p<0.05:
        r_text += " *"
    plt.text(maxval/2.6, maxval/1.2, r_text)
    xticks = plt.xticks()[0]
    yticks = plt.yticks()[0]
    longest_ticks = xticks if len(xticks)>len(yticks) else yticks
    plt.yticks(longest_ticks)
    plt.xticks(longest_ticks)
    plt.ylim(0,  maxval)
    plt.xlim(0, maxval)
    plt.tick_params(**tick_params)

    
plt.tight_layout(pad=2)

#================================================================================#

fig4 = plt.figure(figsize=(13,4))
plt.subplot(131)
sns.boxplot(data=df, hue="Method", y="Mean Localization Error", x="SNR Bins", hue_order=order)
plt.xticks(bins_idc, bin_names)
plt.xlabel("SNR")
plt.ylabel("Mean Localization Error [mm]")
plt.gca().get_legend().remove()

plt.subplot(132)
sns.boxplot(data=df, hue="Method", y="EMD", x="SNR Bins", hue_order=order)
plt.xticks(bins_idc, bin_names)
plt.xlabel("SNR")
plt.gca().get_legend().remove()

plt.subplot(133)
sns.boxplot(data=df, hue="Method", y="Mean Squared Error", x="SNR Bins", hue_order=order)
plt.xticks(bins_idc, bin_names)
plt.xlabel("SNR")
plt.ylim(-0.05e-5, 1e-5)
plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout(pad=2)


fig5 = plt.figure(figsize=(13,4))
plt.subplot(131)
sns.pointplot(data=df, hue="Method", y="Mean Localization Error", x="Number of sources", hue_order=order, errorbar="se")
plt.gca().get_legend().remove()
plt.ylabel("Mean Localization Error [mm]")

plt.subplot(132)
sns.pointplot(data=df, hue="Method", y="EMD", x="Number of sources", hue_order=order, errorbar="se")
plt.gca().get_legend().remove()

plt.subplot(133)
sns.pointplot(data=df, hue="Method", y="Mean Squared Error", x="Number of sources", hue_order=order, errorbar="se")
plt.ylim(-0.05e-5, 3.19e-6)
plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout(pad=2)


# fig1.savefig("figures/accuracy.png", dpi=600)
# fig2.savefig("figures/sparsity_time.png", dpi=600)
# fig3.savefig("figures/extend_estimation.png", dpi=600)
# fig4.savefig("figures/snr.png", dpi=600)
# fig5.savefig("figures/source_number.png", dpi=600)

FLEX
MUSIC
TRAP
MUSIC
Champagne
MCMV
eLOR


# Table: Metrics

In [4]:
df_table = df.copy()

df_table.Method[df.Method=="FLEX\nMUSIC"] = "FLEX-MUSIC"
df_table.Method[df.Method=="TRAP\nMUSIC"] = "TRAP-MUSIC"

df_table = df_table.groupby("Method").median()
df_table = df_table[["Mean Localization Error", "EMD", "Mean Squared Error", "Sparsity", "Time"]]
for col in df_table.columns:
    if not "Squared" in col:
        df_table[col] = df_table[col].round(2)

print(df_table.to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  Mean Localization Error &        EMD &  Mean Squared Error &  Sparsity &  Time \\
Method     &                          &            &                     &           &       \\
\midrule
Champagne  &                     7.73 &  123775.42 &        4.674609e-07 &      6.29 &  2.66 \\
FLEX-MUSIC &                     2.11 &   66291.31 &        4.147606e-07 &      2.36 &  0.35 \\
MCMV       &                    16.25 &  184793.86 &        9.648467e-07 &     30.04 &  1.82 \\
TRAP-MUSIC &                     5.50 &  125233.79 &        5.371027e-07 &      1.73 &  0.32 \\
eLOR       &                    19.05 &  185186.10 &        8.155678e-07 &     28.29 &  0.32 \\
\bottomrule
\end{tabular}



C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\3698118562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table.Method[df.Method=="FLEX\nMUSIC"] = "FLEX-MUSIC"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\3698118562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table.Method[df.Method=="TRAP\nMUSIC"] = "TRAP-MUSIC"
C:\Users\Lukas\AppData\Local\Temp\ipykernel_10732\3698118562.py:12: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended i

# Statistics

In [6]:
df.columns

Index(['Mean Squared Error', 'Normalized Mean Squared Error',
       'Mean Localization Error', 'AUC', 'Corr', 'EMD', 'Sparsity',
       'Sparsity true', 'Active True', 'Active Pred', 'Method', 'Time',
       'Source Extend', 'Sample', 'SNR', 'Number of sources', 'SNR Bins'],
      dtype='object')

In [17]:
import pingouin as pg
dvs = ["Mean Localization Error", "EMD", "Mean Squared Error", "Sparsity"]
df_single = df[df["Source Extend"]=="Single"]
for dv in dvs:
    aov = pg.anova(data=df, dv=dv, between=["Method", "Source Extend"], detailed=True)
    post_hoc_all = pg.pairwise_gameshowell(data=df, dv=dv, between="Method", effsize="cohen")
    post_hoc_single = pg.pairwise_gameshowell(data=df_single, dv=dv, between="Method", effsize="cohen")
    
    print("###########################################")
    print(f"{dv} between Methods:")
    print("###########################################")
    display(aov)
    print("\n Post-hoc All Sources")
    display(post_hoc_all)
    print("\n Post-hoc Single Sources")
    display(post_hoc_single)
    
    print("\n")


###########################################
Mean Localization Error between Methods:
###########################################


,Source,SS,DF,MS,F,p-unc,np2
0,Method,120708.448609,4.0,30177.112152,754.773916,0.000000e+00,0.381918
1,Source Extend,39503.859523,1.0,39503.859523,988.049572,1.103613e-197,0.168206
2,Method * Source Extend,10982.560387,4.0,2745.640097,68.672493,1.217905e-56,0.053227
3,Residual,195350.378283,4886.0,39.981657,NaN,NaN,NaN



 Post-hoc All Sources


,A,B,mean(A),mean(B),diff,se,T,df,pval,cohen
0,Champagne,FLEX\nMUSIC,7.937060,4.556367,3.380693,0.287342,11.765406,1955.673145,0.000000e+00,0.531051
1,Champagne,MCMV,7.937060,14.477167,-6.540106,0.330520,-19.787348,1913.449619,0.000000e+00,-0.885046
2,Champagne,TRAP\nMUSIC,7.937060,7.565487,0.371574,0.338090,1.099039,1796.488466,8.070996e-01,0.050326
3,Champagne,eLOR,7.937060,18.007129,-10.070068,0.285255,-35.302009,1989.393992,0.000000e+00,-1.579197
4,FLEX\nMUSIC,MCMV,4.556367,14.477167,-9.920800,0.324727,-30.551200,1857.490800,0.000000e+00,-1.372804
5,FLEX\nMUSIC,TRAP\nMUSIC,4.556367,7.565487,-3.009120,0.332429,-9.051917,1739.135526,0.000000e+00,-0.417084
6,FLEX\nMUSIC,eLOR,4.556367,18.007129,-13.450762,0.278522,-48.293283,1955.487997,0.000000e+00,-2.181900
7,MCMV,TRAP\nMUSIC,14.477167,7.565487,6.911680,0.370387,18.660674,1925.701291,5.753176e-13,0.848563
8,MCMV,eLOR,14.477167,18.007129,-3.529962,0.322882,-10.932678,1864.708877,4.777290e-13,-0.488924
9,TRAP\nMUSIC,eLOR,7.565487,18.007129,-10.441642,0.330627,-31.581365,1740.785503,0.000000e+00,-1.448645



 Post-hoc Single Sources


...

,Source,SS,DF,MS,F,p-unc,np2
0,Method,762331.932549,4.0,190582.983137,46112.012767,0.000000e+00,0.974183
1,Source Extend,159.805957,1.0,159.805957,38.665437,5.449131e-10,0.007848
2,Method * Source Extend,327.985625,4.0,81.996406,19.839228,3.226972e-16,0.015976
3,Residual,20202.319649,4888.0,4.133044,NaN,NaN,NaN



 Post-hoc All Sources


,A,B,mean(A),mean(B),diff,se,T,df,pval,cohen
0,Champagne,FLEX\nMUSIC,6.717878,2.554060,4.163818,0.106035,39.268294,1218.352405,0.000000e+00,1.745342
1,Champagne,MCMV,6.717878,29.554004,-22.836127,0.131715,-173.374965,1944.314132,7.482903e-13,-7.753564
2,Champagne,TRAP\nMUSIC,6.717878,1.780372,4.937506,0.101880,48.463766,1051.008987,0.000000e+00,2.136739
3,Champagne,eLOR,6.717878,28.197711,-21.479833,0.110484,-194.416546,1395.030096,0.000000e+00,-8.694572
4,FLEX\nMUSIC,MCMV,2.554060,29.554004,-26.999944,0.091436,-295.286616,1301.794818,0.000000e+00,-13.142463
5,FLEX\nMUSIC,TRAP\nMUSIC,2.554060,1.780372,0.773688,0.037300,20.742119,1382.665232,0.000000e+00,0.945369
6,FLEX\nMUSIC,eLOR,2.554060,28.197711,-25.643651,0.056730,-452.027458,1816.681654,0.000000e+00,-20.292686
7,MCMV,TRAP\nMUSIC,29.554004,1.780372,27.773632,0.086584,320.771181,1071.662564,0.000000e+00,14.150949
8,MCMV,eLOR,29.554004,28.197711,1.356293,0.096560,14.046153,1531.999338,0.000000e+00,0.628163
9,TRAP\nMUSIC,eLOR,1.780372,28.197711,-26.417339,0.048523,-544.428228,1245.800550,6.837864e-13,-24.129844



 Post-hoc Single Sources


,A,B,mean(A),mean(B),diff,se,T,df,pval,cohen
0,Champagne,FLEX\nMUSIC,6.726346,1.944817,4.781529,0.161073,29.685405,524.945802,0.000000e+00,1.856968
1,Champagne,MCMV,6.726346,29.202098,-22.475752,0.200564,-112.062856,936.068125,7.260859e-14,-7.087477
2,Champagne,TRAP\nMUSIC,6.726346,1.882309,4.844036,0.160768,30.130655,521.074682,2.797762e-13,1.884492
3,Champagne,eLOR,6.726346,28.139409,-21.413063,0.172520,-124.119613,670.314945,0.000000e+00,-7.850014
4,FLEX\nMUSIC,MCMV,1.944817,29.202098,-27.257281,0.124891,-218.248432,542.837281,0.000000e+00,-13.663330
5,FLEX\nMUSIC,TRAP\nMUSIC,1.944817,1.882309,0.062507,0.034903,1.790905,943.843999,3.794414e-01,0.116087
6,FLEX\nMUSIC,eLOR,1.944817,28.139409,-26.194592,0.071659,-365.547362,641.962708,0.000000e+00,-22.978122
7,MCMV,TRAP\nMUSIC,29.202098,1.882309,27.319789,0.124497,219.442083,536.312958,0.000000e+00,13.734127
8,MCMV,eLOR,29.202098,28.139409,1.062689,0.139342,7.626490,773.413248,3.061995e-13,0.482342
9,TRAP\nMUSIC,eLOR,1.882309,28.139409,-26.257100,0.070969,-369.981102,621.416508,0.000000e+00,-23.238049


# Plot brains

In [ ]:
import pickle as pkl
import mne
import sys; sys.path.insert(0, '../invert')
from invert.util import pos_from_forward
from copy import deepcopy
import pandas as pd

clim=dict(kind="value", pos_lims=(0.0, 0.01, 1))
pp = dict(surface='inflated', hemi='both', background="white", verbose=0, colorbar=False, time_viewer=False)

sim_type = "extended"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_ext, x_test_ext, y_test_ext, sim_info_ext, _, _ = pkl.load(f)

sim_type = "single"
fn = f"evaluation/sim_and_preds_{sim_type}.pkl"
with open(fn, 'rb') as f:
    stc_dict_sing, x_test_sing, y_test_sing, sim_info_sing, _, _ = pkl.load(f)



# Combine
stc_dict = deepcopy(stc_dict_sing)
for key, value in stc_dict_ext.items():
    stc_dict[key].extend(value)
x_test = np.concatenate([x_test_sing, x_test_ext], axis=0)
y_test = np.concatenate([y_test_sing, y_test_ext], axis=0)
sim_info = pd.concat([sim_info_sing, sim_info_ext])


fwd = mne.read_forward_solution("forward_model/64ch_ico3-fwd.fif", verbose=0)
fwd = mne.convert_forward_solution(fwd, force_fixed=True)
pos = pos_from_forward(fwd)
source_model = fwd['src']
vertices = [source_model[0]['vertno'], source_model[1]['vertno']]

samples = [23, 36, 508, 514]
imgs = []
colorbars = []
names = []
for sample in samples:
    
    tmin = 0
    tstep = 1/1000
    subject = "fsaverage"

    evoked = mne.EvokedArray(x_test[sample].T, info, tmin=0)
    # evoked.plot_joint()
    stc = mne.SourceEstimate(y_test[sample].T, vertices, tmin=tmin, tstep=tstep, 
                            subject=subject, verbose=0)
    first_sample = stc.data[:, 0]
    first_sample /= np.max(abs(first_sample))
    stc.data = np.tile(first_sample, (20,1)).T
    brain = stc.plot(**pp, brain_kwargs=dict(title="True"), clim=clim)
    pp["colorbar"] = True
    brain_cb = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
    pp["colorbar"] = False

    img = brain.screenshot()
    colorbar = brain_cb.screenshot()
    brain.close()
    brain_cb.close()
    imgs.append( img )
    colorbars.append( colorbar )
    names.append("Ground Truth")
    
    for solver, stc_list in stc_dict.items():
        first_sample = stc_list[sample].data[:, 0]
        first_sample /= np.max(abs(first_sample))
        stc_list[sample].data = np.tile(first_sample, (20,1)).T
        if solver == "eLORETA" or solver == "MCMV":
            brain = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=dict(kind="value", pos_lims=(0.2, 0.5, 1)))
            pp["colorbar"] = True
            brain_cb = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=dict(kind="value", pos_lims=(0.2, 0.5, 1)))
            pp["colorbar"] = False
        else:
            brain = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
            pp["colorbar"] = True
            brain_cb = stc_list[sample].plot(**pp, brain_kwargs=dict(title=solver), clim=clim)
            pp["colorbar"] = False
            
        img = brain.screenshot()
        colorbar = brain_cb.screenshot()
        
        brain.close()
        brain_cb.close()

        imgs.append( img )
        colorbars.append(colorbar)
        names.append(solver)

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
